In [1]:
from pymongo import MongoClient
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll


db, coll = connectCollection('companies','companies')
db, offices = connectCollection('companies', 'offices')
# db.create_collection("offices")
all_comps = list(coll.find())
all_offices = list(offices.find())

In [83]:
offices[0]

{'_id': ObjectId('5dd13bf6c80571d59fb0a390'),
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-122.394523, 37.764726]},
 'properties': {'name': 'Digg',
  'country': 'USA',
  'company_money_raised': '$45M',
  'foundation_year': 2004}}

In [41]:
target_offices = list(offices.find({"$and" : [{"properties.country":"GBR"}, {"properties.foundation_year": {"$gt": 2009}}, {"properties.company_money_raised" : {"$gte" :"$1M"}} ]}))
geometry = target_offices[0]['geometry']
target_offices[0]

{'_id': ObjectId('5dd13bf7c80571d59fb0ac42'),
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-0.208979, 51.51819]},
 'properties': {'name': 'Pixsta',
  'country': 'GBR',
  'company_money_raised': '£3M',
  'foundation_year': 2010}}

In [39]:
def target_offices(country_code, foundation_year, money_raised):
        return list(offices.find({
            "$and" : [{
                "properties.country":f"{country_code}"}, {
                    "properties.foundation_year": {"$gt": foundation_year}}, {
                        "properties.company_money_raised" : {"$gte" :f"${money_raised}M"}
                        }]
                        } ))
target_offices("GBR", 2008, 1)

[{'_id': ObjectId('5dd13bf7c80571d59fb0ac42'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.208979, 51.51819]},
  'properties': {'name': 'Pixsta',
   'country': 'GBR',
   'company_money_raised': '£3M',
   'foundation_year': 2010}},
 {'_id': ObjectId('5dd13bf7c80571d59fb0b2c3'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-1.574948, 54.703145]},
  'properties': {'name': 'CompareAway',
   'country': 'GBR',
   'company_money_raised': '£500k',
   'foundation_year': 2009}},
 {'_id': ObjectId('5dd13bf9c80571d59fb0c015'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.1020265, 51.5196135]},
  'properties': {'name': 'Shutl',
   'country': 'GBR',
   'company_money_raised': '£7.66M',
   'foundation_year': 2009}}]

In [42]:
def near_offices(geometry, max_distance=2000):
    return list(offices.find({"geometry.coordinates": 
        {"$near" : 
             {
            "$geometry": geometry ,
            "$maxDistance": max_distance
        }}
    }))
near_offices(geometry)

[{'_id': ObjectId('5dd13bf7c80571d59fb0ac42'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.208979, 51.51819]},
  'properties': {'name': 'Pixsta',
   'country': 'GBR',
   'company_money_raised': '£3M',
   'foundation_year': 2010}},
 {'_id': ObjectId('5dd13bf6c80571d59fb0a44d'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.205349, 51.514935]},
  'properties': {'name': 'ConnectMeAnywhere',
   'country': 'GBR',
   'company_money_raised': '$0',
   'foundation_year': 2006}},
 {'_id': ObjectId('5dd13bf7c80571d59fb0ab55'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.209019, 51.5260475]},
  'properties': {'name': 'SOSIUS',
   'country': 'GBR',
   'company_money_raised': '$0',
   'foundation_year': 2007}},
 {'_id': ObjectId('5dd13bf9c80571d59fb0bfa5'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-0.1855258, 51.5174054]},
  'properties': {'name': 'Ruby50',
   'country': 'GBR',
   'company_m

In [146]:
def old_companies(offices, year):
    old_companies = []
    for office in offices: 
        if office['properties']['foundation_year'] < year:
            old_companies.append(office)
    return old_companies

old_companies(near_offices(geometry), 2006)

[]

### ------------

In [129]:
loc

{}

In [111]:
def geocode_request(loc):
    for e in comps:
        for office in e['offices']:
            latitude = office['latitude']
            longitude = office['longitude']
            if latitude == True and longitude == True: 
                url = f"https://geocode.xyz/{latitude},{logitude}?geoit=JSON"
            res = requests.get(url)
                if res.status_code == 200:
                    print(res.url)
            res.json()
            return (data.get("latt",0), data.get("longt",0))
        else:
            raise ValueError(f"Api returned {res.status_code}!=200")

In [112]:
getCoordsFor(coords[0])

https://geocode.xyz/(51.51819,%20-0.208979)?geoit=JSON


('0.00000', '0.00000')

In [4]:
import src.companiesdb as comp
import src.google_api_requests as google
db, offices = comp.connectCollection('companies', 'offices')

ModuleNotFoundError: No module named 'companiesdb'

In [30]:
target_companies = comp.target_offices(offices, 'USA', 2009, 1)
near_count = []
old_count = []
tech_startups_count = []

In [29]:
names = []
result = []
for e in target_companies:
    near = comp.near_offices(offices, e['geometry'])
    near_count.append(len(near))
    old = comp.old_companies(near, 2003)
    old_count.append(len(old))
    tech_startups = comp.tech_startups(near, 2009, target)
    tech_startups_count.append(len(tech_startups))
    res = {
        'reference' : e['properties']['name'],
        'near_companies' : near,
        'old_companies' : old,
        'tech_startups' : tech_startups
    }
    result.append(res)
df = pd.DataFrame({
    'Name' : names,
    'Companies Close' : near_count,
    'Corporations Close' : old_count,
    'Tech Startups' : tech_startups_count
})

ValueError: arrays must all be same length

In [4]:
print(len(target), len(result), len(near_count), len(old_count), len(tech_startups_count))

26 26 26 26 26


In [5]:
result

[{'reference': 'GENWI',
  'near_companies': [{'_id': ObjectId('5dd42889da6c525e8be7c6fa'),
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-111.9035, 33.8171]},
    'properties': {'name': 'GENWI',
     'country': 'USA',
     'company_money_raised': '$7.1M',
     'foundation_year': 2010,
     'category': 'mobile'}}],
  'old_companies': [],
  'tech_startups': [{'_id': ObjectId('5dd42889da6c525e8be7c6fa'),
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-111.9035, 33.8171]},
    'properties': {'name': 'GENWI',
     'country': 'USA',
     'company_money_raised': '$7.1M',
     'foundation_year': 2010,
     'category': 'mobile'}}]},
 {'reference': 'Fixya',
  'near_companies': [{'_id': ObjectId('5dd42889da6c525e8be7c734'),
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-122.323895, 37.566879]},
    'properties': {'name': 'Fixya',
     'country': 'USA',
     'company_money_raised': '$8M',
     'foundation_year': 2013,


In [8]:
target[0]['properties']['name']

'GENWI'

In [22]:
import pandas as pd
headers = ['id', 'name', 'city', 'country', '3 char code', '4 char code', 'latitude', 'longitude', 'a', 'b', 'c', 'd', 'e', 'f']
df = pd.read_csv('./input/airports-extended.csv', names=headers )

df2 = df[df['e'] == 'airport']


geoJSON = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [float(longitude), float(latitude)]
            },
            "properties": {
                "name": e["name"],
                "country": office["country_code"],
                "company_money_raised": e['total_money_raised'],
                "foundation_year": e["founded_year"],
                "category": e['category_code']
            }

(7750, 14)

In [31]:
target_companies

[{'_id': ObjectId('5dd42889da6c525e8be7c6fa'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-111.9035, 33.8171]},
  'properties': {'name': 'GENWI',
   'country': 'USA',
   'company_money_raised': '$7.1M',
   'foundation_year': 2010,
   'category': 'mobile'}},
 {'_id': ObjectId('5dd42889da6c525e8be7c734'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-122.323895, 37.566879]},
  'properties': {'name': 'Fixya',
   'country': 'USA',
   'company_money_raised': '$8M',
   'foundation_year': 2013,
   'category': 'web'}},
 {'_id': ObjectId('5dd42889da6c525e8be7c8ac'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-118.445243, 34.047312]},
  'properties': {'name': 'RazorGator',
   'country': 'USA',
   'company_money_raised': '$58.8M',
   'foundation_year': 2011,
   'category': 'ecommerce'}},
 {'_id': ObjectId('5dd4288ada6c525e8be7cad2'),
  'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-122.3254705, 37.564

In [34]:
####Create Collections from Google API

import src.google_api_requests as google



def starbucks_search(target_companies):
    db, coll = connectCollection('companies', 'companies')
    db.create_collection("starbucks")
    for e in target_companies:
        longitude = e['geometry']['coordinates'][0]
        latitude = e['geometry']['coordinates'][1]
        answer = oogle.nearbyRequest(longitude, latitude, place_type='cafe', keyword='starbucks')
        for e in answer['results']:
            if e['name'] == 'Starbucks':
                db.starbucks.insert_one(e)
                print(f"{longitude}, {latitude})
    
    

-111.9035 33.8171


NameError: name 'nearbyRequest' is not defined

In [35]:
answer = {'html_attributions': [], 'next_page_token': 'CrQCKAEAANUbQ29-iXxfg0d7HEdIHZ2cdmG9h3MRJJONpt1N_tevEltJ394B8mpJgcVAgkCYA5iHEZwCtQUJw6JhqAucqU6Ioww4MtHDJWCK1rxeL6eIyInX-C-8xJChcOqrR4oDOW17qfxzdoMWc32WzWtKpTW5oHYa26h9CjW7PWyPhsiws-e2FwDaRO3hltqX-yIyc120B3E16qJDL_4359VNjOdBlXcQKNajn6ZO2aTkBJ7Bb24x1DDaQAQgYkXVvbvh1sO0y4R_pGsbZea7nxkFrlggPPuImU6xfU6u9XZgGFsw6M3u_xIYmnlmg1GP3Lk3wchLEsOUDN8xNMzK1njtl8_bI1cJx2F3OlRFFUR9FEDD-R4T-5DJJxW6bTMisHFHen5FcrRzRB6CyD2m6617NIYSEGBVU64e0ABRmK2HbBVDlZkaFLljox1oKvFFEP5c0OohUyi_KiRr', 'results': [
        {'geometry': {'location': {'lat': 51.5168329, 'lng': -0.2058693
                }, 'viewport': {'northeast': {'lat': 51.51820667989272, 'lng': -0.2044529701072778
                    }, 'southwest': {'lat': 51.51550702010728, 'lng': -0.2071526298927222
                    }
                }
            }, 'icon': 'https: //maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '9bc805284c9612a7787727603460a1ff18138678', 'name': 'Starbucks', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1920, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116151588495818425911/photos">Alejandro López</a>'], 'photo_reference': 'CmRaAAAA3qXvjJltCArPK1-dPuDUMcjSjHMB1AbCFWGfT3GwpOvGHGEaUcJo79yH8kryeOFNgSpLw25AzIDtNnM_xku9JiChlO_Du05BJNemNkCQG3iK_NFiSOJFaBqfqPa3Lj-LEhAQsx7OZDB2aSZoRzbOUm2XGhRU20NWSnXO8RCf8feKk4Gt_w0EHw', 'width': 1080}], 'place_id': 'ChIJ6c6Jdx4QdkgRAHsmKueikWA', 'plus_code': {'compound_code': 'GQ8V+PM London', 'global_code': '9C3XGQ8V+PM'}, 'price_level': 2, 'rating': 3.6, 'reference': 'ChIJ6c6Jdx4QdkgRAHsmKueikWA', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 210, 'vicinity': '227 Portobello Rd, London'}, {'geometry': {'location': {'lat': 51.5098047, 'lng': -0.1971062}, 'viewport': {'northeast': {'lat': 51.51115302989271, 'lng': -0.1958422701072778}, 'southwest': {'lat': 51.50845337010727, 'lng': -0.1985419298927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '4de1ce7ecef0bf4d090b2aff482ecc20c008808d', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2896, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104771369170657206207/photos">Joey Paysinger</a>'], 'photo_reference': 'CmRaAAAAnVAbJsJxPBlo5UegaiCHMgAGtfgVbH1omfl4v2zjz8dJrg2hdd48lE9PNcPxjjzNbWZMfReR645l1Op1SniDpCL5tCoeDArr6In2viF_9zjFU9eKS066wbdqk1VSuZh2EhAVjiq2mFGL3y3ggiSl5KggGhTL6xa0Nru5QFNxy1H6B2u3ASLAPg', 'width': 3863}], 'place_id': 'ChIJN9p_gfwPdkgROXlJ3NE-ViQ', 'plus_code': {'compound_code': 'GR53+W5 London', 'global_code': '9C3XGR53+W5'}, 'price_level': 2, 'rating': 3.5, 'reference': 'ChIJN9p_gfwPdkgROXlJ3NE-ViQ', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 76, 'vicinity': '64 Notting Hill Gate, London'}, {'geometry': {'location': {'lat': 51.5097984, 'lng': -0.1970972}, 'viewport': {'northeast': {'lat': 51.51114667989273, 'lng': -0.1958376201072778}, 'southwest': {'lat': 51.50844702010728, 'lng': -0.1985372798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '17b22d055f9f0ef5a1931142e03bf4c686b9d565', 'name': 'Starbucks Coffee', 'opening_hours': {}, 'photos': [{'height': 3018, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117624435366291565574/photos">George Austin</a>'], 'photo_reference': 'CmRaAAAAjUoXH9bIB2DVMJYbhY1GNVCnt0V_vH0ILRiSK5lOIDsnyi87UMLFdjutzyWgh0MSKaK2ylQYTTENfAJUI5qY_E0cX7sHuajeLycZtEjmMjm3TRJbpDU4MQ6126Nwlbs7EhBaM0_gXQrRDXll5JgM5-pTGhSDmy72eVWqR4KVqK8lT0IOvLNNQQ', 'width': 3697}], 'place_id': 'ChIJoQGb71UFdkgRNPFB472i_WQ', 'plus_code': {'compound_code': 'GR53+W5 London', 'global_code': '9C3XGR53+W5'}, 'price_level': 2, 'rating': 3.8, 'reference': 'ChIJoQGb71UFdkgRNPFB472i_WQ', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 198, 'vicinity': '26 Pembridge Rd, London'}, {'geometry': {'location': {'lat': 51.507256, 'lng': -0.20582}, 'viewport': {'northeast': {'lat': 51.50854817989273, 'lng': -0.2044292201072778}, 'southwest': {'lat': 51.50584852010728, 'lng': -0.2071288798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '433dc081e2a80f098492b58a8be1af710770916b', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106650612065130026197/photos">Jensen Gooduuyn</a>'], 'photo_reference': 'CmRaAAAABlLNH0ZszwV-T4vKmDtje076i04QC3v1O8-sgPyjGxILMFP4gMivOYyzTTts396UKa18l6WeDIWHnDVw76guHqbCqDHyqxG1SqquPP3u53nggOw8ZOQTcVZPbEFZSR8JEhBZQmFjoqfV-uDPfDj25i46GhTqoAbn6zPy60U5UPwXjOCtK4b4lg', 'width': 3024}], 'place_id': 'ChIJY7H8iOYPdkgR2rPDwq34fCE', 'plus_code': {'compound_code': 'GQ4V+WM London', 'global_code': '9C3XGQ4V+WM'}, 'price_level': 2, 'rating': 4.1, 'reference': 'ChIJY7H8iOYPdkgR2rPDwq34fCE', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 223, 'vicinity': '76 - 78 Holland Park Ave, London'}, {'geometry': {'location': {'lat': 51.5092025, 'lng': -0.2220784}, 'viewport': {'northeast': {'lat': 51.51056912989272, 'lng': -0.2206309701072778}, 'southwest': {'lat': 51.50786947010728, 'lng': -0.2233306298927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': 'd498fa5e7faf35c00a0643ffd550b86505154cf6', 'name': 'Starbucks', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108025572648960848570/photos">Paul Sensational</a>'], 'photo_reference': 'CmRaAAAAlUigzLceEqQsnJ44ZhM3Vc4Q2d_s42e3vuv6cBVt-9QyZ_loH4weESyGhr_3K1ykyUyQEE_mdoowfxTUEhijVCqQ0Cz2t15e8_L62mLZf7ucFiiNJ5ep1YRQ0G9qQ1TbEhCQIEehyT34qkAYE99WT3jWGhTAky41rw8N8WqiHtyXq-n-leW7sQ', 'width': 4032}], 'place_id': 'ChIJTZ98odkPdkgRjArWU3N7ib4', 'plus_code': {'compound_code': 'GQ5H+M5 London', 'global_code': '9C3XGQ5H+M5'}, 'price_level': 2, 'rating': 3.7, 'reference': 'ChIJTZ98odkPdkgRjArWU3N7ib4', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 167, 'vicinity': 'Unit 2122, Ariel Way, London'}, {'geometry': {'location': {'lat': 51.5117122, 'lng': -0.1876108}, 'viewport': {'northeast': {'lat': 51.51301402989272, 'lng': -0.1862482201072778}, 'southwest': {'lat': 51.51031437010728, 'lng': -0.1889478798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '52857c50348814a43b30151f7df921ea633bcf24', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110867876030228629306/photos">Tuomas Olkkola</a>'], 'photo_reference': 'CmRaAAAAxNwX8LcO3ODex_IagoSWG8j6CkBjedYBWYJ3oPDz95lKjAUZT2Wab_G_DM8lm2S1uudTcNd6JMrJEk6q8WxYTHeNYr1B27E5D75QgHzxBpC0cx1pYefQp-jJJFhQAAbXEhB-vphTZl0wcLB2HBx0oTg9GhSGzSS_ENQx8pl2BY8x83kIKaFioA', 'width': 4608}], 'place_id': 'ChIJoQGb71UFdkgRqXoKr4KUick', 'plus_code': {'compound_code': 'GR66+MX London', 'global_code': '9C3XGR66+MX'}, 'price_level': 2, 'rating': 4, 'reference': 'ChIJoQGb71UFdkgRqXoKr4KUick', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 720, 'vicinity': '47-49 Queensway, London'}, {'geometry': {'location': {'lat': 51.5100262, 'lng': -0.2243011}, 'viewport': {'northeast': {'lat': 51.51137482989272, 'lng': -0.2230772201072778}, 'southwest': {'lat': 51.50867517010727, 'lng': -0.2257768798927223}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '3849a5aeefd575a788eadb7c4592179c824320e5', 'name': 'Starbucks Coffee', 'opening_hours': {}, 'photos': [{'height': 498, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100140127863479698731/photos">Adam Fulford</a>'], 'photo_reference': 'CmRaAAAAipMLKB2ZR_CI97kxxdA8rRQ0yDP7LXAZ1SYj5Ap98fzKGR_FsJ_sevk9F8DofFtW-81KIhTCm6x6VbrmfFR62s7d_SOWlKXug5o-aphOcozoR_kOY6o2wtTW0OKlEeznEhDQef3fiqh-vYg9yxKymll6GhQHP6W9Ar3WoDx9HiOrIvRz6kiTsw', 'width': 600}], 'place_id': 'ChIJGYKpUtcPdkgRO559VOGcRdg', 'plus_code': {'compound_code': 'GQ6G+27 London', 'global_code': '9C3XGQ6G+27'}, 'price_level': 2, 'rating': 3.9, 'reference': 'ChIJGYKpUtcPdkgRO559VOGcRdg', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 177, 'vicinity': 'Wood Ln, London'}, {'geometry': {'location': {'lat': 51.5074815, 'lng': -0.2230292}, 'viewport': {'northeast': {'lat': 51.50880972989272, 'lng': -0.2218129701072778}, 'southwest': {'lat': 51.50611007010728, 'lng': -0.2245126298927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'id': 'a1018a965bbe91ef0a7c83caa5c983fc414fd603', 'name': 'Starbucks', 'photos': [{'height': 3968, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104640336228000709309/photos">claudia cristy sánchez</a>'], 'photo_reference': 'CmRaAAAAu2GOoJt2zudrMSBuTnENjfZu6gWYVbr3_w9StT43-_idI9W9jAlH1c41DVnr_cQtA_NeyfuAgfpx_7UQgsKm3o2Y3WxwWn1Zy1aRnsewcA5FSedCcNLwNgNN2i3gK5YXEhA8HoumeqR2tawcOS9BSSrCGhS6-fQmcs4qZDAUx9CaKQpF1YhHiQ', 'width': 2976}], 'place_id': 'ChIJH1jmO34PdkgRnLzJm4t8imU', 'plus_code': {'compound_code': 'GQ4G+XQ London', 'global_code': '9C3XGQ4G+XQ'}, 'rating': 4.2, 'reference': 'ChIJH1jmO34PdkgRnLzJm4t8imU', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 52, 'vicinity': 'Unit 1018a, Ariel Way, London'}, {'geometry': {'location': {'lat': 51.51321429999999, 'lng': -0.227219}, 'viewport': {'northeast': {'lat': 51.51469687989272, 'lng': -0.2256508201072777}, 'southwest': {'lat': 51.51199722010728, 'lng': -0.2283504798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '035d4c22a8c045a3eb41b1537d08639b07bf35dc', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4048, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112656292655697993934/photos">Giovanni Pintus</a>'], 'photo_reference': 'CmRZAAAAYw9vdYWztM8uT18zYDFeD_w4rpzG1Et0hG9UIj6BbiPgbO1zmnl8YoMKJ1dyp-xDh_7XhBxeu4arZox8S6CzlKTMad_rqYZpKlvtxdl6eul23_F9UqNcxtUMo-LnIe4jEhDmehAxjpAnX301I7l_hOUYGhTS6tki9cO4vziGTelvmdLh0dOBKA', 'width': 3036}], 'place_id': 'ChIJeTDUs9IPdkgRzCN-b8YDGv0', 'plus_code': {'compound_code': 'GQ7F+74 London', 'global_code': '9C3XGQ7F+74'}, 'price_level': 2, 'rating': 3.9, 'reference': 'ChIJeTDUs9IPdkgRzCN-b8YDGv0', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 152, 'vicinity': 'British Broadcasting Corporati, 201 Wood Ln, London'}, {'geometry': {'location': {'lat': 51.5045316, 'lng': -0.2195109}, 'viewport': {'northeast': {'lat': 51.50581327989272, 'lng': -0.2181802201072778}, 'southwest': {'lat': 51.50311362010728, 'lng': -0.2208798798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': 'f4f225e17205ff1a85dda3c33fe1260b57ebd3ce', 'name': 'Starbucks', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2610, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104587596115760532487/photos">Ricardo Dias</a>'], 'photo_reference': 'CmRaAAAAPpqvzDs3f0nERFwQxTB-ie3I2tlgPDN6htXF3NCR0txFS02wsChQ7cHk9nn0LtxS-wMJx313dgGssy6p9gKhqTIKczC7_PxBgfF1bmyAGtRbCXFUsiBwg-Y62NRHtp9iEhB5GpnqCFKa-IhTYUEkKjtKGhRRBY051JNE30LrCkAfjwDN_IjScA', 'width': 4640}], 'place_id': 'ChIJU6XlLdsPdkgRkqH-nXI0WBs', 'plus_code': {'compound_code': 'GQ3J+R5 London', 'global_code': '9C3XGQ3J+R5'}, 'price_level': 2, 'rating': 4.1, 'reference': 'ChIJU6XlLdsPdkgRkqH-nXI0WBs', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 195, 'vicinity': '62 Uxbridge Rd, London'}, {'geometry': {'location': {'lat': 51.5194601, 'lng': -0.1795582}, 'viewport': {'northeast': {'lat': 51.52060897989272, 'lng': -0.1782875201072778}, 'southwest': {'lat': 51.51790932010728, 'lng': -0.1809871798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '47177367e684198d33631dc111ac090409b6e941', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1152, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113966983439218053950/photos">Adam Jones</a>'], 'photo_reference': 'CmRaAAAAnX5fULIpcPWgzuByG7jfSTGeu_xltKB_THhkpQhngLzoY1i35weU0j6FGYKwrbOPD9GzOJuPzM0tc5YPqewDXVChQk3iLdHsJx_5AtddgvSlsNioMcPks00rLSPoAFgKEhAsrxJjGm_87ryCKeJJOGV7GhQRqt8T7Yw4SGCww6CAjP5ee9KKCQ', 'width': 2048}], 'place_id': 'ChIJA7sJMqwadkgR24t804XGd9M', 'plus_code': {'compound_code': 'GR9C+Q5 London', 'global_code': '9C3XGR9C+Q5'}, 'price_level': 2, 'rating': 3.9, 'reference': 'ChIJA7sJMqwadkgR24t804XGd9M', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 312, 'vicinity': 'Paddington Central Kiosk, 15 Sheldon Square, London'}, {'geometry': {'location': {'lat': 51.50227289999999, 'lng': -0.1885381}, 'viewport': {'northeast': {'lat': 51.50368992989272, 'lng': -0.1871910701072778}, 'southwest': {'lat': 51.50099027010727, 'lng': -0.1898907298927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '59144ada0167f2b3f1e46d2d0bfa3fbf405b4d2b', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4618, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116899328312293790532/photos">Dylan Rabie</a>'], 'photo_reference': 'CmRaAAAAUKb0otqk7bouWkrc2n08SA3gvT2ka2Bzowh4566FrIO1QT8cJCFTQEBKc27f1RLmlzRzHrtg_Ln14FnotnjU5fyNY7m-_ezwh0bEZSK80XhwIBKdbEaLxydqauHGiqhxEhCGbdG-jVDdoI5F4bQYPwTKGhRB7NOlwjCQzI4TOFpjea9sIFzisQ', 'width': 3464}], 'place_id': 'ChIJM2jO1fcPdkgRnAf3qbL_PBA', 'plus_code': {'compound_code': 'GR26+WH London', 'global_code': '9C3XGR26+WH'}, 'price_level': 2, 'rating': 3.6, 'reference': 'ChIJM2jO1fcPdkgRnAf3qbL_PBA', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 189, 'vicinity': '25A Kensington High St, London'}, {'geometry': {'location': {'lat': 51.51580449999999, 'lng': -0.1764484}, 'viewport': {'northeast': {'lat': 51.51750207989272, 'lng': -0.1747728201072778}, 'southwest': {'lat': 51.51480242010727, 'lng': -0.1774724798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '2fe1a0ada1f5f11a78a869fd1c34c4d7143076b0', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1152, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115557363491693603879/photos">Asuman Güntay</a>'], 'photo_reference': 'CmRaAAAAovEyqP1-OIb4s0TxGexugEHs-gLkysh35mTYC7PrwNwb4nKP2fYCdfaipSNFGf9A4bIa4HiZA9gmUolbpqkeFHlJlhdg7R4kCqE538qIkRYPp_oLn-zshd-q06RYiEkrEhDYIbQGx3ie-kv7DMyYxvlwGhTjQIQpFir96ZE1MKdUfe4St2dE3g', 'width': 2048}], 'place_id': 'ChIJq3PDx41-dkgRyXmyEoqrLTE', 'plus_code': {'compound_code': 'GR8F+8C London', 'global_code': '9C3XGR8F+8C'}, 'price_level': 2, 'rating': 4, 'reference': 'ChIJq3PDx41-dkgRyXmyEoqrLTE', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 146, 'vicinity': 'Heathrow Express Area, Unit 3 The Lawns, London'}, {'geometry': {'location': {'lat': 51.530154, 'lng': -0.186137}, 'viewport': {'northeast': {'lat': 51.53147677989271, 'lng': -0.1848408201072778}, 'southwest': {'lat': 51.52877712010727, 'lng': -0.1875404798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '60915c524e4198197e293add59054df9562b5491', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117266216609555364246/photos">Ehud Bino</a>'], 'photo_reference': 'CmRaAAAAo_ZPYTI1J24lyRmVxes4TNxcDpUJgw5SU_3DmokZDhjvmY84sLV3dDrQqxpPuSvdHnXAQeqxhgDtK76oTnAX4UCXW35gdzfGIzqCF64A1rwOe3jrWYu_z9tJKewUo46cEhCuESws7yDhEwZyCRroLcMQGhSjulHh1PoAhMlxI-WssxRrErPDTA', 'width': 5312}], 'place_id': 'ChIJa2GqMacadkgRfl62bVg9JMw', 'plus_code': {'compound_code': 'GRJ7+3G London', 'global_code': '9C3XGRJ7+3G'}, 'price_level': 2, 'rating': 3.8, 'reference': 'ChIJa2GqMacadkgRfl62bVg9JMw', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 253, 'vicinity': '168-170 Randolph Ave, London'}, {'geometry': {'location': {'lat': 51.53584679999999, 'lng': -0.2052306}, 'viewport': {'northeast': {'lat': 51.53717422989272, 'lng': -0.2039673701072778}, 'southwest': {'lat': 51.53447457010728, 'lng': -0.2066670298927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': 'ae1c2f1a94b3eb6b2a6d44d59dc584738a45d427', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106887172397233664902/photos">Julien Brun</a>'], 'photo_reference': 'CmRaAAAAp9JobA-2MJURglrEZpj4toMJtebToHbWKWGktw9wYLqqrvqqWO7_AOdd0CoDR-M2RSe9uH9IT0IuIW-2IlMsPLr04sh3JJVdmamTrJd8cJs9snXYzte03-B4WI2Tie5hEhCeHq6qbfgiFxo-LEZNgMusGhQUpUnpvAF42zh3A8PQsyz0rcIa0g', 'width': 4032}], 'place_id': 'ChIJJ0HaUhMQdkgRQO05wqNadao', 'plus_code': {'compound_code': 'GQPV+8W London', 'global_code': '9C3XGQPV+8W'}, 'price_level': 2, 'rating': 4.1, 'reference': 'ChIJJ0HaUhMQdkgRQO05wqNadao', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 194, 'vicinity': '89 Salusbury Rd, London'}, {'geometry': {'location': {'lat': 51.4980194, 'lng': -0.1835696}, 'viewport': {'northeast': {'lat': 51.49937117989272, 'lng': -0.1822987201072778}, 'southwest': {'lat': 51.49667152010728, 'lng': -0.1849983798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '0d77de06ef07c6a4a6325e5a09bf01fed8045b7e', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/118356547958849151563/photos">Mohsen Mesgarpour</a>'], 'photo_reference': 'CmRaAAAAELuPcJ9k32t9d-i4IeVAlSGVQ31wLXmnnTTQTmMyLX5Cw58B30kB6wDHWZQrZ8whJ_IIlBy7nGSlTTJMlZMzsLNNhZnpgy6EoXUkistMN-yq--2Hza9K0aRBAUDBmbX1EhCT_RtHYTGsuKjXCPPHcwjTGhQo-M1HSMv8LWBranOrMjX0kHTSRg', 'width': 4032}], 'place_id': 'ChIJrWezwF4FdkgRi9yEi1h_GNM', 'plus_code': {'compound_code': 'FRX8+6H London', 'global_code': '9C3XFRX8+6H'}, 'price_level': 2, 'rating': 3.9, 'reference': 'ChIJrWezwF4FdkgRi9yEi1h_GNM', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 202, 'vicinity': '17-35 Gloucester Rd, London'}, {'geometry': {'location': {'lat': 51.5365863, 'lng': -0.191494}, 'viewport': {'northeast': {'lat': 51.53789727989273, 'lng': -0.1902379201072778}, 'southwest': {'lat': 51.53519762010728, 'lng': -0.1929375798927223}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '941d122d00370ae731a715821854751e6759d431', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116162763608539074562/photos">Nick Miles</a>'], 'photo_reference': 'CmRaAAAAQiByEHP8nDj6YDrA32EiLUI0e3pxMKdLD_ejlHL9fw5Vbm8R9kWnWcKk_HezUhXSKhs5KycFcu0QcJzr4g5VFqCG17Ou3h3cGrPxGa1vx7UkAulasjzRP2pUxqWXO-X6EhAMO7QieEIh-UODPgTlwM4NGhRSs_zzEEfP-slA9I2GyjSIdurWsQ', 'width': 5312}], 'place_id': 'ChIJST8H2XsQdkgRms8oCzTeuVk', 'plus_code': {'compound_code': 'GRP5+JC London', 'global_code': '9C3XGRP5+JC'}, 'price_level': 2, 'rating': 3.8, 'reference': 'ChIJST8H2XsQdkgRms8oCzTeuVk', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 283, 'vicinity': '24-32 Kilburn High Rd, London'}, {'geometry': {'location': {'lat': 51.5189378, 'lng': -0.1688721}, 'viewport': {'northeast': {'lat': 51.52032402989271, 'lng': -0.1674611701072778}, 'southwest': {'lat': 51.51762437010727, 'lng': -0.1701608298927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '213c030e3764f25dda6a62eea1acb567139471f9', 'name': 'Starbucks', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1920, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114204997170065877404/photos">Žaneta Karpova</a>'], 'photo_reference': 'CmRaAAAAj_i6jH5TdWYqPtnaOPhyxedXmqgrbVlYGt1EzD2V5kg2XKWjgdsLlhz62xMqkmrdwkTrfLZxRxm4t-ARXk0kkSgrPVOwhRVMdOkKnQNAHwVZv058isRKGyxKVbIE9Z0AEhCyg9QIVjpz24HkW7hQ8R9lGhSp1zF4Wt6l9bZ0hgW-u_aC4xmqIg', 'width': 1080}], 'place_id': 'ChIJG2-uP7QadkgR6sN3Fcgg2X0', 'plus_code': {'compound_code': 'GR9J+HF London', 'global_code': '9C3XGR9J+HF'}, 'price_level': 2, 'rating': 3.8, 'reference': 'ChIJG2-uP7QadkgR6sN3Fcgg2X0', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 352, 'vicinity': '207 Edgware Rd, London'}, {'geometry': {'location': {'lat': 51.4931962, 'lng': -0.2216074}, 'viewport': {'northeast': {'lat': 51.49447612989272, 'lng': -0.2202513201072778}, 'southwest': {'lat': 51.49177647010728, 'lng': -0.2229509798927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': '231e543b2ca25399cbd2a4cf17f9c5db312534c0', 'name': 'Starbucks', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107497217403289344471/photos">Simon Devlin</a>'], 'photo_reference': 'CmRaAAAAGnk2bQwihiOPCtRQGbZKqRRiFU_bOw6kzb2H2FDISFtGZ-xwgPEjeLkZMUZHIqroomwFgCcIRVkZ9JwbC8N504u2PgfS_rNLZPvI5VaGYa9S-u-CtPG70kWth3Whbbm9EhBr2b0s4cnrhCDXND0MA5Q2GhT37_Dv4gosNy4zaP0DSKIg8n_vTQ', 'width': 5333}], 'place_id': 'ChIJw171bbgPdkgRVQdaKm7qcbA', 'plus_code': {'compound_code': 'FQVH+79 London', 'global_code': '9C3XFQVH+79'}, 'price_level': 2, 'rating': 3.9, 'reference': 'ChIJw171bbgPdkgRVQdaKm7qcbA', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 188, 'vicinity': '200 Hammersmith Rd, London'}, {'geometry': {'location': {'lat': 51.4945272, 'lng': -0.1824408}, 'viewport': {'northeast': {'lat': 51.49586617989272, 'lng': -0.1811407701072778}, 'southwest': {'lat': 51.49316652010727, 'lng': -0.1838404298927222}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png', 'id': 'fa5d23ceb2d0398c172bc138028aa640fc80718e', 'name': 'Starbucks Coffee', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105066049513073389036/photos">Alex Chong</a>'], 'photo_reference': 'CmRaAAAAAtdctMChLTmoqVzAn3hzbZ2xTO0PfZ-yL7OrNB1g0gvjxPNhvYexP9kFB9F0pWyPSDE_DQTIp2RndhjT3RABVNEIH9yHQlnl3SOktG6Lk8cSwZjytqpZzfA5cnVC0TIzEhAoo_yKXfhMY6GW3tJ35VJLGhSeGNH0xBsUZP9gvXbWS0VQDUsjjg', 'width': 2340}], 'place_id': 'ChIJb32pCF4FdkgRWXgxFbqHYwE', 'plus_code': {'compound_code': 'FRV9+R2 London', 'global_code': '9C3XFRV9+R2'}, 'price_level': 2, 'rating': 3.9, 'reference': 'ChIJb32pCF4FdkgRWXgxFbqHYwE', 'scope': 'GOOGLE', 'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 370, 'vicinity': '83 Gloucester Rd, London'}], 'status': 'OK'} 

In [38]:
answer['results'][0]

{'geometry': {'location': {'lat': 51.5168329, 'lng': -0.2058693},
  'viewport': {'northeast': {'lat': 51.51820667989272,
    'lng': -0.2044529701072778},
   'southwest': {'lat': 51.51550702010728, 'lng': -0.2071526298927222}}},
 'icon': 'https: //maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png',
 'id': '9bc805284c9612a7787727603460a1ff18138678',
 'name': 'Starbucks',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1920,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116151588495818425911/photos">Alejandro López</a>'],
   'photo_reference': 'CmRaAAAA3qXvjJltCArPK1-dPuDUMcjSjHMB1AbCFWGfT3GwpOvGHGEaUcJo79yH8kryeOFNgSpLw25AzIDtNnM_xku9JiChlO_Du05BJNemNkCQG3iK_NFiSOJFaBqfqPa3Lj-LEhAQsx7OZDB2aSZoRzbOUm2XGhRU20NWSnXO8RCf8feKk4Gt_w0EHw',
   'width': 1080}],
 'place_id': 'ChIJ6c6Jdx4QdkgRAHsmKueikWA',
 'plus_code': {'compound_code': 'GQ8V+PM London',
  'global_code': '9C3XGQ8V+PM'},
 'price_level': 2,
 'rating': 3.6,
 'reference': 'ChIJ6c6Jdx4QdkgRAHs

In [8]:
import src.companiesdb as comp
import requests
import folium

In [50]:
# TODO: Center the map with the midde point of all coordinates
# https://fiona.readthedocs.io/en/latest/manual.html
db, offices = comp.connectCollection('companies', 'offices')
target = comp.target_offices(offices, 'USA', 2009, 1)
m = folium.Map()
def plotTargetOffices(target_companies):
    for company in target_companies:
        folium.Marker(location=company['geometry']['coordinates'][::-1],
                     radius=2,
                     icon=folium.Icon(icon='cloud', color='blue')).add_to(m)

db, starbucks = comp.connectCollection('companies', 'starbucks')
starbucks_list = list(starbucks.find())
def plotStarbucks(starbucks_list):
    for starbucks in starbucks_list:
        folium.Marker(location=[starbucks['geometry']['location']['lat'], 
                                starbucks['geometry']['location']['lng']],
                     radius=2,
                     icon=folium.Icon(color='green')).add_to(m)

In [46]:
starbucks_list

[{'_id': ObjectId('5dd59e59e9842528f515b62f'),
  'geometry': {'location': {'lat': 33.8217146, 'lng': -111.9228886},
   'viewport': {'northeast': {'lat': 33.82303872989272,
     'lng': -111.9217050701073},
    'southwest': {'lat': 33.82033907010727, 'lng': -111.9244047298927}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png',
  'id': '740ed935f775077877dc90c45b9c794797cd2b8e',
  'name': 'Starbucks',
  'opening_hours': {'open_now': True},
  'place_id': 'ChIJmf3ZyMh-K4cR9X_HC7QcOV4',
  'plus_code': {'compound_code': 'R3CG+MR Carefree, Arizona',
   'global_code': '855CR3CG+MR'},
  'price_level': 2,
  'rating': 3.8,
  'reference': 'ChIJmf3ZyMh-K4cR9X_HC7QcOV4',
  'scope': 'GOOGLE',
  'types': ['cafe', 'food', 'point_of_interest', 'store', 'establishment'],
  'user_ratings_total': 4,
  'vicinity': '36889 N Tom Darlington Dr, Scottsdale'},
 {'_id': ObjectId('5dd59e59e9842528f515b630'),
  'geometry': {'location': {'lat': 33.781056, 'lng': -111.922309},
   'viewport':

In [48]:
plotStarbucks(starbucks_list)

In [52]:
plotTargetOffices(target)

In [53]:
m

In [56]:
import json
json = json.dumps(target)
with open("target_offices.json","w") as file:
        file.write(json)

TypeError: Object of type 'ObjectId' is not JSON serializable

In [59]:
def target_offices(offices, country_code, foundation_year, money_raised):
    target_offices = offices.find({
        "$and": [
            {"properties.country": f"{country_code}"},
            {"properties.foundation_year": {"$gt": foundation_year}},
            {"properties.company_money_raised": {"$gte": f"${money_raised}M"}}
        ]
    })
    return target_offices
